# WSI - Ćwiczenie 2

*Autor: Maksymilian Nowak*

### Cel ćwiczenia

Celem ćwiczenia jest zaimplementowanie algorytmu ewolucyjnego:
- z selekcją turniejową ($k=2$)
- krzyżowaniem jednopunktowym
- mutacją gaussowską
- sukcesją generacyjną

Należy również wykorzystać ten algorytm do znalezienia minimum sumy funkcji $f_1+f_2$, gdzie: 
$$f_1(x_1,y_1)=(x_1^2+y_1-11)^2+(x_1+y_1^2-7)^2$$
$$f_2(x_2,y_2)=2*x_2^2+1.05*x_2^4+\frac{x_2^6}{6}+x_2y_2+y_2^2$$
dla dziedziny $D=[-5,5]\times[-5,5]$, oraz zbadać wpływ dystrybucji populacji początkowej, prawdopodobieństwa mutacji oraz prawdopodobieństwa krzyżowania na współrzędne znalezionych minimów funkcji $f_1$.

## Implementacja algorytmu ewolucyjnego

Do implementacji algorytmu przyjąłem następujące założenia:
- Funkcja celu ma następującą postać: $g(x_1, x_2, y_1, y_2)=f_1(x_1,y_1)+f_2(x_2,y_2)$
- Osobnik jest postaci $(x_1,x_2,y_1,y_2)$
- Kryterium stopu w algorytmie będzie maksymalna liczba pokoleń

Na początku zdefiniuję badane funkcje oraz funkcję celu, będącą ich sumą:

In [1]:
def f1(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

def f2(x, y):
    return 2*x**2 + 1.05*x**4 + (1/6)*x**6 + x*y + y**2

def g(x1, x2, y1, y2):
    return f1(x1, y1) + f2(x2, y2)

Pozostałymi hiperparametrami algorytmu będą:
- prawdopodobieństwo mutacji
- wariancja używana do obliczania siły mutacji 
- prawdopodobieństwo krzyżowania
- wartość kary za wyjście poza planszę (czyli dziedzinę $D$)

Przed zaimplementowaniem algorytmu wyznaczę funkcję oceny - wartością zwracaną przez tę funkcję będzie wynik funkcji celu (im mniejszy, tym lepsza ocena). Dodatkowo kara za wyjście poza dziedzinę funkcji przez osobnika to dodanie 1000 do wyniku działania funkcji oceny.

In [2]:
def rating(x_1, x_2, y_1, y_2, penalty_value):
    penalty = 0
    domain_conditions = [
        -5 <= x_1 <= 5,
        -5 <= x_2 <= 5,
        -5 <= y_1 <= 5,
        -5 <= y_2 <= 5
    ]
    if not all(domain_conditions):
        penalty += penalty_value
    return g(x_1, x_2, y_1, y_2) + penalty

In [ ]:
import numpy as np
import random

def evolutionary(function, population, crossover_prob, variance, mutation_prob, penalty, max_generations):
    generation = 0
    ratings = []
    # Ocena pierwszego pokolenia
    for individual in population:
        ratings.append(rating(individual[0], individual[1], individual[2], individual[3], penalty))
    while generation < max_generations:
        # Selekcja turniejowa
        selected = []
        for i, individual in enumerate(population):
            opponents = selected.copy().pop(i)
            opponent = random.choice(opponents)
            if rating(individual[0], individual[1], individual[2], individual[3], penalty) < rating(opponent[0], opponent[1], opponent[2], opponent[3], penalty):
                selected.append(individual)
            else:
                selected.append(opponent)
        # Krzyżowanie jednopunktowe
        crossed = []
        crossover_value = random.uniform(0, 1)
        for i in range(0, len(selected), 2):
            parents = random.sample(selected, 2)
            if crossover_value < crossover_prob:
                crossing_point = random.randint(len(selected[0]))
                children = [
                    parents[0][:crossing_point] + parents[1][crossing_point:],
                    parents[1][:crossing_point] + parents[0][crossing_point:]
                ]
            else:
                children = parents
        crossed.append(children[0])
        crossed.append(children[1])
        # Mutacja
        mutated = []
        for individual in crossed:
            mutation_value = random.uniform(0, 1)
            if mutation_value < mutation_prob:
                mutated.append(individual + variance * np.random.normal(0, 1, len(individual)))
            else:
                mutated.append(individual)
        # Ocena pokolenia
        ratings = []
        for individual in mutated:
            ratings.append(rating(individual[0], individual[1], individual[2], individual[3], penalty))
        # Zastąpienie populacji
        population = mutated
        generation += 1
    return population, ratings
